In [56]:
!pip install pandas matplotlib seaborn sqlite3 numpy pandas

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for sqlite3


In [57]:
!pip install pandas openpyxl



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
!pip install matplotlib


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3



In [60]:
import pandas as pd
import sqlite3
# Load the Excel file
df = pd.read_excel("Amazon Sale Report.xlsx")  # Ensure this file is in the same folder

# Connect to SQLite DB (creates file if it doesn't exist)
conn = sqlite3.connect("amazon_sales.db")

# Write the data into a table named 'sales'
df.to_sql("sales", conn, if_exists="replace", index=False)

print(" Excel file imported into SQLite database as 'sales' table.")
conn.close()

 Excel file imported into SQLite database as 'sales' table.


In [61]:
# Inspect first rows of the DataFrame
print(df.head())

   index             Order ID       Date                        Status  \
0      0  405-8078784-5731545 2022-04-30                     Cancelled   
1      1  171-9198151-1101146 2022-04-30  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146 2022-04-30                       Shipped   
3      3  403-9615377-8133951 2022-04-30                     Cancelled   
4      4  407-1069790-7240320 2022-04-30                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in          Expedited  JNE3671  JNE3671-TU-XXXL   

        Category  ... currency  Amount    ship-city   ship-state  \
0            Set  ...      INR

In [62]:
print(df.head(10))
print(df.shape)
print(df.dtypes)

   index             Order ID       Date                        Status  \
0      0  405-8078784-5731545 2022-04-30                     Cancelled   
1      1  171-9198151-1101146 2022-04-30  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146 2022-04-30                       Shipped   
3      3  403-9615377-8133951 2022-04-30                     Cancelled   
4      4  407-1069790-7240320 2022-04-30                       Shipped   
5      5  404-1490984-4578765 2022-04-30                       Shipped   
6      6  408-5748499-6859555 2022-04-30                       Shipped   
7      7  406-7807733-3785945 2022-04-30  Shipped - Delivered to Buyer   
8      8  407-5443024-5233168 2022-04-30                     Cancelled   
9      9  402-4393761-0311520 2022-04-30                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style                  SKU  \
0   Merchant      Amazon.in           Standard   SET389       SET389-KR-NP-S   
1   Merchant      Amazon.

In [63]:
# Step 1: View missing values
print("🔍 Missing values per column:")
print(df.isnull().sum())

🔍 Missing values per column:
index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
Unnamed: 22           49050
dtype: int64


### Initial Descriptive Statistics

In [64]:
print(df.describe())


               index                           Date            Qty  \
count  128975.000000                         128975  128975.000000   
mean    64487.000000  2022-05-12 11:49:27.951928576       0.904431   
min         0.000000            2022-03-31 00:00:00       0.000000   
25%     32243.500000            2022-04-20 00:00:00       1.000000   
50%     64487.000000            2022-05-10 00:00:00       1.000000   
75%     96730.500000            2022-06-04 00:00:00       1.000000   
max    128974.000000            2022-06-29 00:00:00      15.000000   
std     37232.019822                            NaN       0.313354   

              Amount  ship-postal-code  Unnamed: 22  
count  121180.000000     128942.000000      79925.0  
mean      648.561465     463966.236509          0.0  
min         0.000000     110001.000000          0.0  
25%       449.000000     382421.000000          0.0  
50%       605.000000     500033.000000          0.0  
75%       788.000000     600024.000000       

In [65]:
# Step 2: Display original column names
print("Original columns:", df.columns.tolist())

Original columns: ['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22']


In [66]:
# Step 3: Drop rows with critical missing values
df = df.dropna(subset=["Order ID", "Date", "Qty", "Amount"])


In [67]:
# Step 4: Rename columns for SQL compatibility
df.rename(columns={
    "Order ID": "Order_ID",
    "Date": "Order_Date",
    "Sales Channel ": "Sales_Channel",
    "ship-service-level": "Ship_Service_Level",
    "Courier Status": "Courier_Status",
    "Qty": "Quantity_Ordered",
    "Amount": "Total_Revenue",
    "ship-city": "Ship_City",
    "ship-state": "Ship_State",
    "ship-postal-code": "Ship_Postal_Code",
    "ship-country": "Ship_Country"
}, inplace=True)



In [68]:
# Step 5: Drop unnamed columns (like 'Unnamed: 22' or similar)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]



In [69]:
# Step 6: Convert data types
df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors='coerce')
df["Quantity_Ordered"] = pd.to_numeric(df["Quantity_Ordered"], errors='coerce')
df["Total_Revenue"] = pd.to_numeric(df["Total_Revenue"], errors='coerce')



In [70]:
# Step 7: Remove duplicate rows
df.drop_duplicates(inplace=True)



In [71]:
# Step 8: Export cleaned DataFrame to SQLite
conn = sqlite3.connect("amazon_sales.db")
df.to_sql("sales", conn, if_exists="replace", index=False)
conn.close()

print(" Data cleaning completed and saved to SQLite as 'sales' table.")

 Data cleaning completed and saved to SQLite as 'sales' table.
